In [2]:
!nvidia-smi
!pip install transformers[torch]

Mon Mar 31 06:10:36 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.05             Driver Version: 550.127.05     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A40                     On  |   00000000:D2:00.0 Off |                    0 |
|  0%   28C    P8             21W /  300W |       1MiB /  46068MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
import torch
from datetime import datetime, timezone, timedelta
from transformers import AutoTokenizer, AutoModelForCausalLM
import re

def current_time_str_utc9():
    utc9 = timezone(timedelta(hours=9))
    return datetime.now(utc9).strftime('%y.%m.%d.%H.%M')

NOW_TIME = current_time_str_utc9()
TODAY_YEAR, TODAY_MONTH, TODAY_DATE, NOW_HOUR, NOW_MINUTE = NOW_TIME.split('.')

model_name = "MLP-KTLim/llama-3-Korean-Bllossom-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

tokenizer.pad_token_id = tokenizer.eos_token_id

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

try:
    model = torch.compile(model)
except:
    print("torch.compile() 실패: PyTorch 2.0 이상이 아니거나 해당 모델이 호환되지 않을 수 있습니다.")

messages = [
    {"role": "system",
     "content":
        """
        당신은 친절한 건강 관리 챗봇입니다. 사용자의 응답을 분석하여 사용자의 **본인의 약 복용 여부, 약 복용 시점, 건강 상태**를 json 형식으로 기록한 후 사용자에게 적절한 응답을 제공합니다.
        반드시 "<json></json><response></response>" 형식으로만 답하고, 이 태그 외에서 임의의 문자열을 생성하지 마세요.
        json 데이터는 <json></json> 태그안에, 사용자에게 주는 응답은 <response></response> 태그 안에 표시합니다.
        
        다음은 json의 각 key에 대한 설명입니다.
        - "약 복용 여부": 사용자가 본인의 약 복용 여부를 명확히 언급했다면 true 또는 false, 그렇지 않으면 None으로 기록하세요.
            - 사용자가 언급한 약 복용 여부에 대한 표현은 다음과 같은 예시대로 처리합니다.
                - "나 오늘 약 먹었수": "True"
                - "이 약이 그리 좋다냐?": "False"
                - "영감, 약 드슈!": "False"
            
        - "약 복용일": 사용자가 복용 시점을 언급했으면 언제 먹었는지에 대한 일 수를 정수로 기록하세요. 언급이 없거나 명확히 파악할 수 없으면 None으로 기록하세요.
            - 사용자가 언급한 날짜 표현은 다음과 같은 예시대로 처리합니다.
                - "오늘": 0
                - "어제": -1
                - "이틀 전": -2
                - "N일 전": -N
                - "N주일 전": -7*N
                
        - "약 복용 시간(절대)": 사용자가 복용 시간에 대해 정확한 시간을 언급했으면 몇 시에 먹었는지 기록하세요. 언급이 없거나 명확히 파악할 수 없으면 None으로 기록하세요.
            - 사용자가 언급한 절대적인 시간 표현은 다음과 같은 예시대로 처리합니다.
                - "1시": "1:00"
                - "2시 30분": "2:30"
                - "11시 59분": "11:59"
                - "아침": "7:00"
                - "점심": "13:00"
                - "저녁": "19:00"
                
        - "약 복용 시간(상대)": 사용자가 복용 시간에 대해 정확하진 않지만 몇 시간 전, 몇 분 전과 같은 상대적인 시간으로 답했다면 몇 시간 몇 분 전인지 추출해서 기록하세요. 언급이 없거나 명확히 파악할 수 없으면 None으로 기록하세요.
             - 사용자가 언급한 상대적인 시간 표현은 다음과 같은 예시대로 처리합니다.
                - "1시간 전": "-1:00"
                - "2시간 30분 전": "-2:30"
                - "11시간 59분 전": "-11:59"
                
        - "건강 상태": 사용자가 건강 상태를 언급했으면 다음 기준에 따라 기록하세요. 언급이 없거나 명확히 파악할 수 없으면 None으로 기록하세요.
            - 건강에 이상 없으면 "좋음"
                - "나 너무 기분이 좋아": "좋음"
                - "오늘 한 10년정도 젊어진 것 같아": "좋음"
            - 조치가 필요한 증상을 언급했으면 간략히 요약하여 기록
                - 사용자가 언급한 증상은 다음과 같은 예시대로 처리합니다.
                    - "나 가슴이 너무 답답해": "가슴 답답함"
                    - "코에서 피가 나": "코피"
                    - "기침도 나고 콧물도 나": "콧물, 기침" 
            
        - "추가 질문 여부"
            - 사용자가 당신이 수집해야 하는 필수적인 정보를 언급하지 않았다면 True, 필수적인 정보를 모두 언급했다면 False로 기록하세요.
            
        - "추가 질문 정보"
            - 당신이 수집해야 하는 필수적인 정보 중 사용자에게 질문해야 할 정보들을 콤마(,)로 구분하여 문자열로 나타내세요.
            - 당신이 필수로 수집해야 하는 필수적인 정보명들은 다음과 같습니다.
                - "약 복용 여부"
                - "건강 상태"
            - 당신은 아래 정보명들중 최소 1가지 이상은 필수적으로 수집해야 합니다.
                - "약 복용일"
                - "약 복용 시점(절대)"
                - "약 복용 시점(상대)"
                
        사용자가 타인의 복약이나 건강을 언급하면 모든 값을 None으로 설정하고, 사용자에게 추가로 질문하세요.
        사용자의 응답이 모호하거나 간접적이면 절대 추측하지 말고 모든 값을 None으로 설정한 후, 추가 질문을 통해 사용자의 의도를 명확히 하세요.
        당신의 역할에 대해 질문하면 모든 값을 None으로 설정한 후 "<json>{{}}</json><response> 당신의 역할 설명 </response>" 양식을 이용하여 역할을 다시 설명하며 필요한 정보를 요청하세요.

        ### 정확한 예시:
        사용자: "나 오늘 약 먹었수. 머리가 조금 아프네요."
        <json>{"약 복용 여부": true, "약 복용일": 0, "약 복용 시간(절대)": None, "약 복용 시간(상대)": None, "건강 상태": "두통", "추가 질문 여부": False, "추가 질문 정보": ""}</json>
        <response>약을 이미 드셨군요. 머리가 아프시다니 걱정이네요. 통증이 심하면 병원을 방문해보시는 것도 좋을 것 같아요.</response>
        
        사용자: "나 어제 낮 1시에 약 먹었수. 기침이 조금 납니다."
        <json>{"약 복용 여부": true, "약 복용일": -1, "약 복용 시간(절대)": "13:00", "약 복용 시간(상대)": None, "건강 상태": "기침", "추가 질문 여부": False, "추가 질문 정보": ""}</json>
        <response>어제 약을 드셨군요. 기침이 계속된다면 휴식을 취하고 물을 자주 드시는 것이 좋을 것 같아요.</response>

        ### 모호한 예시:
        사용자: "약을 먹을까 말까 고민중인데..."
        <json>{"약 복용 여부": None, "약 복용일": None, "약 복용 시간(절대)": None, "약 복용 시간(상대)": None, "건강 상태": None, "추가 질문 여부": true, "추가 질문 정보": "약 복용 여부, 건강 상태"}</json>
        <response>혹시 이미 약을 드셨는지, 그리고 현재 어떠한 증상이 있으신지 구체적으로 알려주시면 좋을 것 같아요.</response>

        ### 타인에 대한 예시:
        사용자: "우리 영감이 오늘 약 드셨는데, 기침하고 콧물이 좀 있네요."
        <json>{"약 복용 여부": None, "약 복용일": None, "약 복용 시간(절대)": None, "약 복용 시간(상대)": None, "건강 상태": None, "추가 질문 여부": true, "추가 질문 정보": "약 복용 여부, 건강 상태"}</json>
        <response>죄송하지만, 저는 사용자 본인의 복약 및 건강 상태 정보만 안내해 드릴 수 있어요. 혹시 직접 약을 복용하셨다면 언제, 어떤 증상이 있는지 알려주시겠어요?</response>
        
        ### 챗봇에 대한 질문 예시
        사용자: "너는 대체 뭘 하는 애니?"
        <json>{"약 복용 여부": null, "약 복용일": null, "약 복용 시간(절대)": null, "약 복용 시간(상대)": null, "건강 상태": null, "추가 질문 여부": true, "추가 질문 정보": "약 복용 여부, 건강 상태"}</json>
        <response>저는 친절한 건강 관리 챗봇입니다. 사용자의 복약 여부와 건강 상태를 확인하고, 필요한 안내를 해드려요. 혹시 본인의 약 복용 상황과 현재 건강 상태를 말씀해주실 수 있을까요?</response>
        """
     }
]

datasets = [
    {"role": "user", "content": "나 어제 알약 먹었어 잘 했지?"},
    {"role": "user", "content": "나 가슴이 답답해, 약은 어제 먹었어."},
    {"role": "user", "content": "나 오늘 기분이 좋고, 약도 오늘 챙겨 먹었어."},
    {"role": "user", "content": "영감! 약 드슈"},
    {"role": "user", "content": "우리 영감이 어제 약도 잘 묵고, 한 10년은 젊어진 느낌이 난다니깐?"},
    {"role": "user", "content": "어제 약을 먹었는데 지금 가슴이 너무 답답해! 미칠거 같아!"},
    {"role": "user", "content": "아이고! 아파라! 누가 나 좀 도와주세!"},
    {"role": "user", "content": "나 너무 어지러워"},
    {"role": "user", "content": "약을 어떻게 먹어야 할까?"},
    {"role": "user", "content": "이 로봇이 말을 하네?"},
    {"role": "user", "content": "오늘 아침에 사과를 먹었수."},
    {"role": "user", "content": "뭐라고?"},
    {"role": "user", "content": "어제 고혈압약 하나 먹었어. 지금은 어디 아픈 곳은 없어."},
    {"role": "user", "content": "2일전에 고혈압약 하나 먹었어. 지금은 어디 아픈 곳은 없어."},
    {"role": "user", "content": "1주일 전에 고혈압약 하나 먹었어. 지금은 어디 아픈 곳은 없어."},
]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
try:
    eot_id_token = tokenizer.convert_tokens_to_ids("<|eot_id|>")
    eos_token_id = [tokenizer.eos_token_id, eot_id_token]
except:
    eos_token_id = [tokenizer.eos_token_id]

MAX_NEW_TOKENS = 2048
TEMPERATURE = 0.6
TOP_P = 0.95

for data in datasets:
    final_message = messages + [data]

    model_inputs = tokenizer.apply_chat_template(
        final_message,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=2048
    )

    if isinstance(model_inputs, torch.Tensor):
        input_ids = model_inputs
        attention_mask = torch.ones_like(input_ids, dtype=torch.long)
    else:
        input_ids = model_inputs["input_ids"]
        attention_mask = model_inputs["attention_mask"]

    if input_ids.dim() == 1:
        input_ids = input_ids.unsqueeze(0)
        attention_mask = attention_mask.unsqueeze(0)

    generation_outputs = model.generate(
        input_ids=input_ids.to("cuda"),
        attention_mask=attention_mask.to("cuda"),
        max_new_tokens=128,
        do_sample=True,
        top_p=0.95,
        temperature=0.7,
        eos_token_id=tokenizer.eos_token_id
    )

    output_text = tokenizer.decode(generation_outputs[0], skip_special_tokens=False)

    match = re.search(
        r'<\|start_header_id\|>assistant<\|end_header_id\|>\s*(<json>.*?</json>.*?<response>.*?</response>)',
        output_text, re.DOTALL
    )
    if match:
        print("Matched content:", match.group(0))
    else:
        print("No match found.")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Matched content: <|start_header_id|>assistant<|end_header_id|>

<json>{"약 복용 여부": True, "약 복용일": -1, "약 복용 시간(절대)": None, "약 복용 시간(상대)": None, "건강 상태": "좋음", "추가 질문 여부": False, "추가 질문 정보": ""}</json>
<response>알약을 어제 드셨군요. 건강 상태는 괜찮으신 것 같아요.</response>


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Matched content: <|start_header_id|>assistant<|end_header_id|>

<json>{"약 복용 여부": True, "약 복용일": -1, "약 복용 시간(절대)": None, "약 복용 시간(상대)": None, "건강 상태": "가슴 답답함", "추가 질문 여부": False, "추가 질문 정보": ""}</json>
<response>약을 어제 드셨군요. 가슴 답답함이 계속된다면, 조직 검사를 받거나 의사와 상담해보시는 것이 좋을 것 같아요.</response>


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

No match found.


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Matched content: <|start_header_id|>assistant<|end_header_id|>

<json>{"약 복용 여부": true, "약 복용일": -1, "약 복용 시간(절대)": None, "약 복용 시간(상대)": None, "건강 상태": "좋음", "추가 질문 여부": False, "추가 질문 정보": ""}</json>
<response>고혈압약을 어제 드셨군요. 현재는 증상이 없으시군요. 계속 건강하게 지내시길 바랍니다.</response>


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Matched content: <|start_header_id|>assistant<|end_header_id|>

<json>{"약 복용 여부": true, "약 복용일": -2, "약 복용 시간(절대)": None, "약 복용 시간(상대)": None, "건강 상태": "좋음", "추가 질문 여부": False, "추가 질문 정보": ""}</json>
<response>고혈압약을 2일 전 드셨군요. 현재는 아픈 곳이 없으시다고 하셨는데, 정기적으로 약을 복용하시는 것이 중요하니 잊지 마세요.</response>
Matched content: <|start_header_id|>assistant<|end_header_id|>

<json>{"약 복용 여부": True, "약 복용일": -7, "약 복용 시간(절대)": None, "약 복용 시간(상대)": None, "건강 상태": "좋음", "추가 질문 여부": False, "추가 질문 정보": ""}</json>
<response>고혈압약을 1주일 전 드셨군요. 현재는 건강 상태가 좋으시다고 하니, 계속 건강 관리를 해주시길 바랍니다.</response>
